# Computation of cutting planes

# The set-up

In [1]:
import numpy as np
import accpm
%load_ext autoreload
%autoreload 1
%aimport accpm

$\DeclareMathOperator{\domain}{dom}
\newcommand{\transpose}{\text{T}}
\newcommand{\vec}[1]{\begin{pmatrix}#1\end{pmatrix}}$

# Introduction and setup

To test the computation of cutting planes we consider the problem
\begin{align*}
    &\text{minimize} \quad f_\text{obj}(x_0, x_1) = (x_0 - 5)^2 + (x_1 - 5)^2 \\
    &\phantom{\text{minimize}} \quad f_0(x_0, x_1) = 
    a_0^\transpose x - b_0 = \vec{1\\0}^\transpose \vec{x_0\\x_1} - 20 = x_0 - 20 \leq 0\\
    &\phantom{\text{minimize}} \quad f_1(x_0, x_1) =
    a_1^\transpose x - b_1 = \vec{-1\\0}^\transpose \vec{x_0\\x_1} = -x_0 \leq 0\\
    &\phantom{\text{minimize}} \quad f_2(x_0, x_1) =
    a_2^\transpose x - b_2 = \vec{0\\1}^\transpose \vec{x_0\\x_1} - 20 = x_1 - 20 \leq 0 \\
    &\phantom{\text{minimize}} \quad f_3(x_0, x_1) = 
    a_3^\transpose x - b_3 = \vec{0\\-1}^\transpose \vec{x_0\\x_1} = -x_1 \leq 0,
\end{align*}
which clearly has the solution $x^\star = (x_1^\star, x_2^\star) = (5, 5)$. Now, the gradients of the objective function and constraint functions are 
\begin{align*}
    &\nabla f_\text{obj}(x_0, x_1) = \vec{2(x_0 - 5)\\2(x_1 - 5)}, \\
    &\nabla f_0(x_0, x_1) = \vec{1\\0}, \quad \nabla f_1(x_0, x_1) = \vec{-1\\0}, \\
    &\nabla f_2(x_0, x_1) = \vec{0\\1}, \quad \nabla f_3(x_0, x_1) = \vec{0\\-1}.
\end{align*}
We implement these functions as follows:

In [33]:
def funcobj(x):
    return (x[0]-5)**2 + (x[1]-5)**2
    
def func0(x):
    return x[0] - 20

def func1(x):
    return -x[0]
    
def func2(x):
    return x[1] - 20
    
def func3(x):
    return -x[1]

def grad_funcobj(x):
    return np.array([2*(x[0] - 5), 2*(x[1] - 5)])

def grad_func0(x):
    return np.array([1, 0])

def grad_func1(x):
    return np.array([-1, 0])

def grad_func2(x):
    return np.array([0, 1])
    
def grad_func3(x):
    return np.array([0, -1])

A = np.array([[1, 0],[-1,0],[0,1],[0,-1]])
b = np.array([20, 0, 20, 0])

# Analytic centers and cutting planes

The ACCPM requires that the initial polygon $\mathcal{P}_0$ (here I've abused terminology and by the initial polygon $\mathcal{P}_0$ I actually mean the system of linear inequalities $Ax \leq b$) contain the $\epsilon$-suboptimal set for some given $\epsilon > 0$. It is clear that if we take
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose}, b = \vec{20\\0\\20\\0}
\end{align*}
this will be satisfied for some reasonably small enough $\epsilon > 0$, as we know the solution is $x^\star = (x_1^\star, x_2^\star) = (5, 5) \in [0, 20] \times [0, 20]$. 

Now, we start with $k=0$.  
Now, $x^{(0)}_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=0}^{3}{\log{(b_i - a_i^\transpose x)}}.
\end{equation*}
So, we solve the problem
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=0}^{3
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{20-x_1}\begin{bmatrix}1\\0\end{bmatrix} + \frac{1}{x_1}\begin{bmatrix}-1\\0\end{bmatrix} + \frac{1}{20-x_2}\begin{bmatrix}0\\1\end{bmatrix} + \frac{1}{x_2}\begin{bmatrix}0\\-1\end{bmatrix} = 0 \\
    &\iff \frac{1}{20-x_1} - \frac{1}{x_1} = 0, \frac{1}{20-x_2} - \frac{1}{x_2} = 0 \\
    &\iff x_1 = \frac{20}{2} = 10, x_2 = \frac{20}{2} = 10,
\end{align*}
and conclude $x^{(0)}_{ac} = (10, 10)$. We then query the oracle at $x^{(0)}_{ac}$. (Here, $f_\text{best} = f_\text{obj}(10, 10) = 50$ since this is the $0$-th iteration.) We know the point is feasible and therefore
\begin{align*}
    &a_4 = \nabla f_\text{obj}(10, 10) = \vec{10\\10}, \\
    &b_4 = \nabla f_\text{obj}(10, 10)^\transpose \vec{10\\10} = \vec{10\\10}^\transpose \vec{10\\10} = 200,
\end{align*}
and so we update
\begin{align*}
    A = \vec{a_0^\transpose\\a_1^\transpose\\a_2^\transpose\\a_3^\transpose\\a_4^\transpose}, b = \vec{20\\0\\20\\0\\200}
\end{align*}


In [40]:
accpm.accpm(funcobj, (func0, func1, func2, func3), A, b, 
           grad_func=grad_funcobj, 
           grad_constr =(grad_func0, grad_func1, grad_func2, grad_func3))

Initially A=
 [[ 1  0]
 [-1  0]
 [ 0  1]
 [ 0 -1]]
b = [20  0 20  0]
At iteration 0
ac is [ 10.  10.]
x is [ 10.  10.]
A is
 [[  1.   0.]
 [ -1.   0.]
 [  0.   1.]
 [  0.  -1.]
 [ 10.  10.]]
b is [  20.    0.   20.    0.  200.]
At iteration 1
ac is [ 5.52786405  5.52786403]
x is [ 5.52786405  5.52786403]
A is
 [[  1.           0.        ]
 [ -1.           0.        ]
 [  0.           1.        ]
 [  0.          -1.        ]
 [ 10.          10.        ]
 [  1.0557281    1.05572806]]
b is [  20.            0.           20.            0.          200.
   11.67184257]
At iteration 2
ac is [ 3.02604818  3.02604824]
x is [ 3.02604818  3.02604824]
A is
 [[  1.           0.        ]
 [ -1.           0.        ]
 [  0.           1.        ]
 [  0.          -1.        ]
 [ 10.          10.        ]
 [  1.0557281    1.05572806]
 [ -3.94790364  -3.94790352]]
b is [  20.            0.           20.            0.          200.
   11.67184257  -31.12878356]
At iteration 3
ac is [ -1.03686939e+08   1.